This notebook contains code that performs a dynamic sentiment analysis based on emojis.

Emojis are extracted from tweets, and the tweet text is used to predict the emoji.

In [ ]:
from extract_tweets import get_raw_tweets
import pickle

raw_tweets = get_raw_tweets('BackLAOut',5e7)
raw_tweets += get_raw_tweets('BackNYOut',5e7)

with open('raw_BackLA_BackNY', 'wb') as fp:
    pickle.dump(raw_tweets, fp)

In [ ]:
from extract_tweets import clean_tweets
import pickle
with open('raw_BackLA_BackNY', 'rb') as fp:
    raw_tweets = [tweet for tweet in pickle.load(fp)]
    
cleaned_tweets = clean_tweets(raw_tweets)

with open('clean_BackLA_BackNY', 'wb') as fp:
    pickle.dump(cleaned_tweets, fp)
    
len(cleaned_tweets)

In [ ]:
import random
import emoji
from itertools import chain
from collections import OrderedDict
from extract_tweets import get__X_and__y
import pickle
with open('clean_BackLA_BackNY', 'rb') as fp:
    cleaned_tweets = [tweet for tweet in pickle.load(fp)]

random.shuffle(cleaned_tweets)

emojis = emoji.core.unicode_codes.UNICODE_EMOJI
emoji_sentiment1 = dict(zip(emojis, [0]*len(emojis)))
emoji_sentiment2 = \
OrderedDict([('☹', -1),('👺', -1),('👿', -1),('💔', -1),('💙', -1),('🖤', -1),('😑', -1),
             ('😒', -1),('😓', -1),('😔', -1),('😕', -1),('😖', -1),('😞', -1),('😟', -1),
             ('😠', -1),('😡', -1),('😢', -1),('😣', -1),('😤', -1),('😥', -1),('😦', -1),
             ('😧', -1),('😨', -1),('😩', -1),('😫', -1),('😭', -1),('😰', -1),('😱', -1),
             ('😲', -1),('😳', -1),('😵', -1),('😾', -1),('😿', -1),('🙁', -1),('🙄', -1),
             ('🤒', -1),('🤕', -1),('🤢', -1),('🤥', -1),('🤬', -1),('🤮', -1),('🥵', -1),
             ('🥺', -1),('🤦', -1),('🤦🏻', -1),('🤦🏼', -1),('🤦🏽', -1),('🤦🏾', -1),('🤦🏿', -1),
             
             ('☺',  1),('♥',  1),('❣',  1),('❤',  1),('🐱',  1),('👅',  1),('💑',  1),
             ('💓',  1),('💕',  1),('💖',  1),('💗',  1),('💘',  1),('💚',  1),('💛',  1),
             ('💜',  1),('💝',  1),('💞',  1),('💟',  1),('😀',  1),('😁',  1),('😂',  1),
             ('😃',  1),('😄',  1),('😅',  1),('😆',  1),('😇',  1),('😈',  1),('😉',  1),
             ('😊',  1),('😋',  1),('😌',  1),('😍',  1),('😎',  1),('😏',  1),('😗',  1),
             ('😘',  1),('😙',  1),('😚',  1),('😛',  1),('😜',  1),('😝',  1),('😸',  1),
             ('😺',  1),('😻',  1),('😼',  1),('😽',  1),('🙂',  1),('🙃',  1),('🤍',  1),
             ('🤎',  1),('🤑',  1),('🤗',  1),('🤛',  1),('🤛🏻',  1),('🤛🏼',  1),('🤛🏽',  1),
             ('🤛🏾',  1),('🤛🏿',  1),('🤜',  1),('🤜🏻',  1),('🤜🏼',  1),('🤜🏽',  1),('🤜🏾',  1),
             ('🤜🏿',  1),('🤠',  1),('🤡',  1),('🤤',  1),('🤪',  1),('🤭',  1),('🥰',  1),
             ('🥳',  1),('🥴',  1),('🧡',  1)])
for e in emojis:
    if e in emoji_sentiment2:
        del emoji_sentiment1[e]
emoji_sentiment = OrderedDict(chain(emoji_sentiment1.items(),emoji_sentiment2.items()))

with open('emoji_sentiment', 'wb') as fp:
    pickle.dump(emoji_sentiment, fp)

In [ ]:
from extract_tweets import get__X_and__y
import pickle
with open('emoji_sentiment', 'rb') as fp:
    emoji_sentiment = pickle.load(fp)

_X, _y = get__X_and__y(cleaned_tweets, emoji_sentiment)

with open('_X', 'wb') as fp:
    pickle.dump(_X, fp)
with open('_y', 'wb') as fp:
    pickle.dump(_y, fp)

In [ ]:
from collections import Counter, OrderedDict
from math import ceil
import numpy as np
from functools import reduce
import pickle
with open('_X', 'rb') as fp:
    _X = [tweet for tweet in pickle.load(fp)]
with open('_y', 'rb') as fp:
    _y = pickle.load(fp)

batch_size = 64

def split_tr_val_te(X):
    percent_tr = 1/3
    percent_val = 1/3

    tr_size = batch_size*ceil(percent_tr*len(X)/batch_size)
    val_size = ceil(percent_val*(len(X)-tr_size)/(1-percent_tr))
    te_size = len(X) - tr_size - val_size

    X_val = X[:val_size]
    X_tr = X[val_size:-te_size]
    X_te = X[-te_size:]
    print(len(X),len(X_tr)/len(X),len(X_val)/len(X),len(X_te)/len(X),len(X_tr)/batch_size)
    return X_tr, X_val, X_te

_X_tr,_X_val,_X_te = split_tr_val_te(_X)
_y_tr,_y_val,_y_te = split_tr_val_te(_y)

def split_tweets_into_words(_X):
    return [[word for word in tweet.split(' ')] for tweet in _X]

_X_tr = split_tweets_into_words(_X_tr)
_X_val = split_tweets_into_words(_X_val)
_X_te = split_tweets_into_words(_X_te)

emoji_counts = Counter()
for i in _y_tr:
    emoji_counts[i] += 1
emoji_counts, {k:v/sum(emoji_counts.values()) for k,v in emoji_counts.items()}

In [ ]:
# get word counts and vocabulary size
runonsentence = ' '.join([' '.join(tweet) for tweet in _X_tr]+
                         [' '.join(tweet) for tweet in _X_val]+
                         [' '.join(tweet) for tweet in _X_te]).split(' ')

words = np.unique(runonsentence)
def get_counts_adv(_X):
    d1 = dict(zip(words,[0]*len(words)))
    d2 = dict(zip(words,[0]*len(words)))
    d3 = dict(zip(words,[0]*len(words)))
    
    for x in _X:
        for w in x:
            d1[w] += 1
    m1 = np.mean(list(d1.values()))
    sd1 = np.std(list(d1.values()))
    print(m1,sd1)
    
    for x in _X:
        d = Counter()
        for w in x:
            d[w] = 1
        for w in d:
            d2[w] += d[w]
    m2 = np.mean(list(d2.values()))
    sd2 = np.std(list(d2.values()))
    print(m2,sd2)
    
    fdfdsf = list(reduce(lambda i, j: set(i) | set(j), _X))
    for w in fdfdsf:
        d3[w] = (d1[w] - m1) / sd1 + (d2[w] - m2) / sd2

    return d3

wc_tr = get_counts_adv(_X_tr)
wc_val = get_counts_adv(_X_val)
wc_te = get_counts_adv(_X_te)

for w in words:
    if wc_tr[w]*wc_val[w]*wc_te[w]==0:
        del wc_tr[w]
        del wc_val[w]
        del wc_te[w]

In [ ]:
def get_most_frequent_keys(d,n):
    a = np.array(list(d.values()),dtype='float')
    a -= np.nanmin(a)
    N = min(n,len(d))
    inds = a >= 0.99 * min(a[np.argpartition(a,-N)][-N:])
    d = dict(np.array(list(d.items()))[inds])
    for k in d:
        d[k] = float(d[k])
    odct = OrderedDict(sorted(d.items(),key=lambda x:x[1], reverse=True))
    return odct

vocab = get_most_frequent_keys(wc_tr,5000)
print(len(vocab))

def get_counts(i):
    d = Counter()
    for j in i:
        d[j] += 1
    return d

runonsentence = [w if w in vocab else '<UNK>' for w in runonsentence]
word_counts = get_counts(runonsentence)
    
assert(min(list(word_counts.values())) > 1)

word_counts = OrderedDict(sorted(word_counts.items()), key=lambda x: x[0], reverse=True)

vocabulary_size = len(word_counts)
vocabulary_size, word_counts['<UNK>'], _X_tr[:10]

In [ ]:
T=np.array([(0,len(x)) for x in _X_tr]+
           [(1,len(x)) for x in _X_val]+
           [(2,len(x)) for x in _X_te])
T=np.vstack([T.T,np.arange(len(T))]).T
tr=len(_X_tr)
val=len(_X_val)
T[:,2][tr:] -= tr
T[:,2][tr+val:] -= val
_X_tr[0:2],_X_val[0:2],_X_te[0:2],T[0:2],T[tr:tr+2],T[tr+val:tr+val+2]

In [ ]:
i = 0
for t in T:
    for j in range(t[1]):
        if t[0] == 0:
            _X_tr[t[2]][j] = runonsentence[i]
        elif t[0] == 1:
            _X_val[t[2]][j] = runonsentence[i]
        else:
            _X_te[t[2]][j] = runonsentence[i]
        i += 1
_X_tr[0:2],_X_val[0:2],_X_te[0:2]

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
import pickle
with open('emoji_sentiment', 'rb') as fp:
    emoji_sentiment = pickle.load(fp)

def convert_to_id(_X):
    return [[word2id[word] for word in tweet] for tweet in _X]

def convert_to_one_hot(_y):
    #return enc.transform(np.array(_y).reshape(-1,1)).toarray()
    _y = np.array(_y) - np.array(_y).min()
    _y = _y * (len(np.unique(_y))-1) / max(_y)
    print(_y)
    return to_categorical(_y, num_classes=len(np.unique(list(emoji_sentiment.values()))))

word2id = {}
for i,word in enumerate(word_counts):
    word2id[word] = i

max_words = max([len(x) for x in _X])
X_tr= sequence.pad_sequences(convert_to_id(_X_tr), maxlen=max_words)
X_val = sequence.pad_sequences(convert_to_id(_X_val), maxlen=max_words)
X_te = sequence.pad_sequences(convert_to_id(_X_te), maxlen=max_words)

y_tr = convert_to_one_hot(_y_tr)
y_val = convert_to_one_hot(_y_val)
y_te = convert_to_one_hot(_y_te)

X_tr[:20], y_tr[:20]

This concludes the data grooming. Run the analysis!

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.arange(len(y_tr[0])),
                                                  y=np.argmax(y_tr,axis=1))
# class_weights = class_weight.compute_class_weight('balanced',
#                                                   classes=np.arange(len(y_tr[0]))-1,
#                                                   y=_y_tr)

print(np.sum(y_tr,axis=0)*class_weights)

class_weights = {i:w for i,w in zip(range(len(y_tr[0])),class_weights)}

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model=Sequential()

embedding_size=32
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))

n_neurons = ceil(np.sqrt(0.15 * len(X_tr)))
model.add(Bidirectional(LSTM(n_neurons, return_sequences=True),
                        backward_layer=LSTM(n_neurons, return_sequences=True,
                                            activation='relu', go_backwards=True)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons))
model.add(Dropout(0.2))
if len(class_weights) == 2:
    model.add(Dense(2, activation='sigmoid'))
else:
    model.add(Dense(len(class_weights), activation='softmax'))

model.summary()

In [ ]:
loss = 'binary_crossentropy' if len(class_weights) == 2 else 'categorical_crossentropy'
model.compile(loss=loss, optimizer='Nadam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'temp.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
num_epochs = 25
model.fit(X_tr, y_tr, validation_data=(X_val, y_val), batch_size=batch_size,
          epochs=num_epochs, class_weight=class_weights, callbacks=[checkpoint])

In [ ]:
model.load_weights(filepath)
yhat_te = model.predict(X_te)

In [ ]:
score=np.argmax(yhat_te,axis=1)==np.argmax(y_te,axis=1)
yhat_te,y_te,score,np.mean(score)